In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
from pathlib import Path

pd.set_option('future.no_silent_downcasting', True)

BASE_DIR = Path("/Users/konstantin/Desktop/Рабочий стол — Konstantin’s MacBook Air/IT/PyCharm/dashboard_project/app")


CARDS_DATA_PATH = Path(os.getenv("CARDS_DATA_PATH", BASE_DIR / "data/2025 год/cards prod/"))
BRELOKI_DATA_PATH = Path(os.getenv("BRELOKI_DATA_PATH", BASE_DIR / "data/2025 год/table prod/"))
LOCAL_OUTPUT_PATH = Path(os.getenv("LOCAL_OUTPUT_PATH", BASE_DIR / "data/"))
LOCAL_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)


def num_z(x, y):
    # Преобразуем дату из строки в datetime
    date_obj = pd.to_datetime(y, dayfirst=True)
    cutoff_date = datetime(2025, 3, 1)  # 01.03.2025

    x_str = str(x).strip().replace('Y', 'У').replace('.0', '')

    if date_obj < cutoff_date:
        # Если строка уже начинается с '25УП' или '24УП', возвращаем её
        if x_str.startswith('25УП') or x_str.startswith('24УП'):
            return x_str
        else:
            # Обработка для строк длиной 5 символов
            if len(x_str) == 5:
                return f'24УП-{x_str.zfill(6)}'
            # Обработка для строк длиной 4 символа
            elif len(x_str) < 5:
                return f'25УП-{x_str.zfill(6)}'
            else:
                # Можно добавить обработку для других случаев или вернуть исходное значение
                return x_str
    else:
        # Для дат после cutoff_date можно определить другую логику или вернуть исходное значение
        if '25УП' not in x_str:
            return f'25УП-{x_str.zfill(6)}'

        return x_str

def reorder_cols(df):
    cols = df.columns.tolist()
    base_cols = ['Дата', '№ заказа', 'Наименование', 'Количество', 'Доп. Расход чипов', 'Излишки']
    new_order = [col for col in base_cols if col in cols]
    brak_columns = [col for col in cols if col.startswith('Виды брака_')]
    new_order.extend(brak_columns)
    remaining_cols = [col for col in cols if col not in new_order]
    new_order.extend(remaining_cols)
    return df[new_order]

def add_nomenklature(df):
    order_file = LOCAL_OUTPUT_PATH / 'order_money.xlsx'
    if not order_file.exists():
        return df

    order_num = pd.read_excel(order_file, header=3)

    order_num['Заказ на производство'] = order_num['Заказ на производство'].apply(lambda x: str(x)[22:33])
    df = df.merge(order_num[['Заказ на производство', 'Номенклатура, Вид номенклатуры', 'Цена', 'Количество']],
                  left_on='№ заказа', right_on='Заказ на производство', how='left', suffixes=('_i', '_a_n'))

    if "Номенклатура_a_n" in df.columns:
        df["Номенклатура"] = df["Номенклатура_a_n"].fillna("Отсутствует информация")
        df.drop(columns=["Номенклатура_a_n"], inplace=True)
    elif "Номенклатура" in df.columns:
        df["Номенклатура"] = df["Номенклатура"].fillna("Отсутствует информация")
    else:
        df["Номенклатура"] = "Отсутствует информация"

    return df

def cards_data_collect():
    full_data = pd.DataFrame()

    for file_path in CARDS_DATA_PATH.rglob("*.xls*"):
        try:
            xls = pd.ExcelFile(file_path)
            sheet_names = [s for s in xls.sheet_names if s[-1].isdigit()]
            for sheet in tqdm(sheet_names, desc=f"Обработка карточек {file_path.name}"):
                tmp_data = pd.read_excel(file_path, sheet_name=sheet)
                tmp_data = tmp_data.iloc[:, :tmp_data.columns.get_loc('Брак без видимых причин') + 1]

                for i in range(len(tmp_data.columns)):
                    if 'Unnamed' in tmp_data.columns[i]:
                        tmp_data.rename(columns={tmp_data.columns[i]: f'{tmp_data.columns[i-1]}'}, inplace=True)
                for i in range(len(tmp_data.columns)):
                    if pd.notna(tmp_data.iloc[0, i]):
                        tmp_data.columns.values[i] = f'{tmp_data.columns[i]}_{tmp_data.iloc[0, i]}'

                tmp_data = tmp_data[1:]
                # tmp_data = tmp_data.loc[~tmp_data['Дата'].isin(['Количество брака за день', 'Цех rfid'])]
                tmp_data['Дата'] = pd.to_datetime(tmp_data['Дата'], dayfirst=True, errors='coerce').ffill()
                tmp_data = tmp_data.loc[~tmp_data['№ заказа'].isna()]
                tmp_data = tmp_data.loc[pd.notna(tmp_data['№ заказа'])]
                tmp_data = tmp_data.fillna(0)
                tmp_data['Количество'] = pd.to_numeric(tmp_data['Количество'], downcast='integer')

                full_data = pd.concat([full_data, tmp_data], ignore_index=True)
        except Exception as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")

    full_data['Участок'] = 'Карточный цех'


    full_data['№ заказа'] = full_data.apply(
        lambda row: num_z(row['№ заказа'], row['Дата']),
        axis=1
    )

    return full_data

def breloki_data_collect():
    full_data = pd.DataFrame()

    for file_path in BRELOKI_DATA_PATH.rglob("*.xls*"):
        try:
            df = pd.read_excel(file_path, sheet_name='Брак 2025 год')
            df = df.rename(columns={'Причина брака': 'Виды брака', 'Наименование тиража': 'Наименование'})

            for i in range(len(df.columns)):
                if 'Unnamed' in df.columns[i]:
                    df.rename(columns={df.columns[i]: f'{df.columns[i-1]}'}, inplace=True)
            for i in range(len(df.columns)):
                if pd.notna(df.iloc[0, i]):
                    df.columns.values[i] = f'{df.columns[i]}_{df.iloc[0, i]}'

            df = df[1:]
            df['Дата'] = pd.to_datetime(df['Дата'], dayfirst=True, errors='coerce')

            df = df.rename(columns={
                'Виды брака_Персонализация': 'Виды брака_Персонализационный дефект',
                'Виды брака_Трещина': 'Виды брака_Трещины',
                'Виды брака_Грязь': 'Виды брака_Инородные включения'
            })

            defect_cols = [c for c in df.columns if c.startswith('Виды брака_')] + ['Излишки']

            df['№ заказа'] = df['№ заказа'].astype(str).apply(lambda x: x.strip().replace('.0', ''))

            for col in defect_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

            df['кол-во брака'] = df[defect_cols].sum(axis=1)
            df['Участок'] = 'Цех заливки'
            df = df.loc[~df['Дата'].isna()].fillna(0)

            full_data = pd.concat([full_data, df], ignore_index=True)
        except Exception as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")

    full_data['№ заказа'] = full_data.apply(
        lambda row: num_z(row['№ заказа'], row['Дата']),
        axis=1
    )

    return full_data

def save_full_data(df: pd.DataFrame):
    out_xlsx = LOCAL_OUTPUT_PATH / 'card_full_data.xlsx'
    out_csv = LOCAL_OUTPUT_PATH / 'card_full_data.csv'

    df.to_excel(out_xlsx, index=False)
    df.to_csv(out_csv, index=False)
    print(f'''Сохранено: 
{out_xlsx}
{out_csv}''')

def main():
    print("=== Поиск файлов ===")

    card_files = list(CARDS_DATA_PATH.rglob("*.xls*"))
    breloki_files = list(BRELOKI_DATA_PATH.rglob("*.xls*"))

    if card_files:
        print("Файлы карточек:")
        for f in card_files:
            print("  -", f.relative_to(BASE_DIR))
    else:
        print("❌ Файлы карточек не найдены")

    if breloki_files:
        print("Файлы брелоков:")
        for f in breloki_files:
            print("  -", f.relative_to(BASE_DIR))
    else:
        print("❌ Файлы брелоков не найдены")

    print("\n=== Сборка данных ===")

    cards_df = cards_data_collect() if card_files else pd.DataFrame()
    breloki_df = breloki_data_collect() if breloki_files else pd.DataFrame()

    if cards_df.empty and breloki_df.empty:
        print("⚠️ Нет данных для обработки — выходим.")
        return

    full_df = pd.concat([cards_df, breloki_df], ignore_index=True).fillna(0)

    display(full_df)

    # добавляем номенклатуру
    full_df = add_nomenklature(full_df)#.fillna(0)
    #
    # # заменяем "25УП" на "24УП" там, где нет инфы по номенклатуре
    # mask = full_df['Номенклатура'] == 'Отсутствует информация'
    # full_df.loc[mask, '№ заказа'] = (
    #     full_df.loc[mask, '№ заказа'].astype(str).str.replace("25УП", "24УП", regex=False)
    # )
    # full_df = full_df.drop(columns=['Количество_a_n', 'Номенклатура', 'Заказ на производство', 'Цена'], axis=1)
    # # повторяем попытку сопоставления
    # full_df = add_nomenklature(full_df)#.fillna(0)

    # считаем браки
    defect_columns = [col for col in full_df.columns if col.startswith('Виды брака_')]
    #
    for col in defect_columns:
        full_df[col] = pd.to_numeric(full_df[col], errors="coerce").fillna(0)

    full_df['кол-во брака'] = full_df[defect_columns].sum(axis=1)

    # объединяем количество
    if {'Количество_i', 'Количество_a_n'}.issubset(full_df.columns):
        full_df['Количество_i'] = full_df[['Количество_i', 'Количество_a_n']].max(axis=1)
        full_df = full_df.drop('Количество_a_n', axis=1).rename(columns={'Количество_i': 'Количество'})


    # % брака
    full_df["кол-во брака"] = pd.to_numeric(full_df["кол-во брака"], errors="coerce").fillna(0)
    full_df["Количество"] = pd.to_numeric(full_df["Количество"], errors="coerce").fillna(0)

    full_df["% брака"] = (
            (full_df["кол-во брака"] / full_df["Количество"].replace(0, pd.NA)) * 100
    ).fillna(0).astype(float).round(2)

    # стоимость брака
    if 'Цена' in full_df.columns:
        full_df['Стоимость брака'] = full_df['кол-во брака'] * full_df['Цена']
    # else:
    #     full_df['Стоимость брака'] = 0

    # финальная перестановка колонок
    full_df = reorder_cols(full_df)
    #
    full_df = full_df.drop(columns=['СКМ', 'Заказ на производство'], axis=1)
    full_df = full_df.fillna(0)

    save_full_data(full_df)


if __name__ == '__main__':
    main()


=== Поиск файлов ===
Файлы карточек:
  - data/2025 год/cards prod/3 квартал/Сентябрь (1).xlsx
  - data/2025 год/cards prod/3 квартал/Июль (1).xlsx
  - data/2025 год/cards prod/3 квартал/Август (1).xlsx
  - data/2025 год/cards prod/2 квартал/Июнь.xlsx
  - data/2025 год/cards prod/2 квартал/Май.xlsx
  - data/2025 год/cards prod/2 квартал/Апрель.xlsx
  - data/2025 год/cards prod/1 квартал/Март.xlsx
  - data/2025 год/cards prod/1 квартал/Февраль.xlsx
  - data/2025 год/cards prod/1 квартал/Январь.xlsx
Файлы брелоков:
  - data/2025 год/table prod/% брака_брелоки (10).xlsx

=== Сборка данных ===


Обработка карточек Январь.xlsx: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:01<00:00, 16.35it/s]


,Дата,№ заказа,Наименование,Количество,% брака,Виды брака_Трещины,Виды брака_Разнотон,Виды брака_Царапины,Виды брака_Инородные включения,Виды брака_Персонализационный дефект,...,СКМ,Виды брака_Разнотон (фрезеровка-имплантация),Участок,Доп. Расход чипов,Излишки,Виды брака_Чип Х,Виды брака_Другой ЧИП,Виды брака_Ламинация,Виды брака_Заливка,кол-во брака
0,2025-09-01,25УП-25уп-009471,скм,1387.0,0.025955,0,0,12,14,0,...,0,0,Карточный цех,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-09-01,25УП-25уп-009488,скм,2.0,0.500000,0,0,0,1,0,...,0,0,Карточный цех,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-09-01,25УП-25уп-009801,скм,841.0,0.023781,0,0,1,1,0,...,0,0,Карточный цех,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-09-01,25УП-24УП-010878,скм,356.0,0.002809,0,0,0,1,0,...,0,0,Карточный цех,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-09-01,25УП-009350,скм,486.0,0.008230,0,0,0,1,1,...,0,0,Карточный цех,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6749,2025-09-12,25УП-009508,ТСН Эмеральд,200.0,0.022624,0.0,0,0,2.0,0.0,...,0,0,Цех заливки,21.0,16.0,2.0,0.0,0.0,0.0,21.0
6750,2025-09-12,25УП-008435,SE1K7B Лесные жители Медведь,300.0,0.003086,0.0,0,0,0.0,0.0,...,0,0,Цех заливки,24.0,23.0,1.0,0.0,0.0,0.0,24.0
6751,2025-09-12,25УП-009195,SE1K7B Лесные жители Лось,60.0,0.013889,0.0,0,0,0.0,0.0,...,0,0,Цех заливки,12.0,11.0,1.0,0.0,0.0,0.0,12.0
6752,2025-09-12,25УП-008361,Самоцветы Изумруд 2 чипа АТА 5577+Classic 7B,60.0,0.152778,0.0,0,0,0.0,0.0,...,0,0,Цех заливки,12.0,1.0,10.0,1.0,0.0,0.0,12.0


Сохранено: 
/Users/konstantin/Desktop/Рабочий стол — Konstantin’s MacBook Air/IT/PyCharm/dashboard_project/app/data/card_full_data.xlsx
/Users/konstantin/Desktop/Рабочий стол — Konstantin’s MacBook Air/IT/PyCharm/dashboard_project/app/data/card_full_data.csv


In [2]:
pd.read_excel('/Users/konstantin/Desktop/Рабочий стол — Konstantin’s MacBook Air/IT/PyCharm/dashboard_project/app/data/order_money.xlsx')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Параметры:,NaN,Период: 01.01.2024 - 15.09.2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Заказ на производство дата,NaN,NaN,Заказ на производство номер,Заказ на производство,NaN,Заказ клиента,"Номенклатура, Вид номенклатуры",Характеристика,Цена,Количество,Сумма
3,09.08.2024 14:42:31,NaN,NaN,24УП-008860,Заказ на производство 24УП-008860 от 09.08.202...,NaN,Заказ клиента 24УП-006934 от 07.08.2024 10:34:18,"Метка ISBC Labels 50х50 UHF, UCODE8, PET adhes...",NaN,13.64,1100,15004
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISBC Products,NaN,NaN,2820,324972.5
...,...,...,...,...,...,...,...,...,...,...,...,...
17393,20.06.2024 8:47:46,NaN,NaN,24УП-007044,Заказ на производство 24УП-007044 от 20.06.202...,NaN,Заказ клиента 24УП-001643 от 22.02.2024 12:22:23,Кабель ISBC ANT.C12-A UHF Antenna Cable 12m Lo...,NaN,3831.8,2,7663.6
17394,05.07.2024 10:42:04,NaN,NaN,24УП-007614,Заказ на производство 24УП-007614 от 05.07.202...,NaN,Заказ клиента 24УП-005684 от 25.06.2024 12:36:24,Кабель ISBC ANT.C12-A UHF Antenna Cable 12m Lo...,NaN,3658.33,12,43899.96
17395,10.09.2024 12:15:23,NaN,NaN,24УП-009973,Заказ на производство 24УП-009973 от 10.09.202...,NaN,Заказ клиента 24УП-007977 от 05.09.2024 12:14:29,Кабель ISBC ANT.C12-A UHF Antenna Cable 12m Lo...,NaN,4600,1,4600
17396,17.10.2024 14:52:42,NaN,NaN,24УП-011565,Заказ на производство 24УП-011565 от 17.10.202...,NaN,Заказ клиента 24УП-009369 от 17.10.2024 14:51:19,Кабель ISBC ANT.C12-A UHF Antenna Cable 12m Lo...,NaN,5680,2,11360


In [23]:
def plot_line_total(data, period, save_path='line_total.png'):
    start_date = pd.to_datetime(period[0], dayfirst=True)
    end_date = pd.to_datetime(period[1], dayfirst=True)

    data = data.copy()
    data['Дата'] = pd.to_datetime(data['Дата'], errors='coerce')

    filtered = data[(data['Дата'] >= start_date) & (data['Дата'] <= end_date)]
    grouped = filtered.groupby('Дата')['Количество'].sum()

    fig = Figure(figsize=(20, 7))
    ax = fig.subplots()
    ax.plot(grouped.index, grouped.values, linestyle='-', color='blue', marker='o')
    ax.set_title(f'Вырубка карт: {start_date.strftime("%d.%m.%y")} — {end_date.strftime("%d.%m.%y")}')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Количество')
    ax.grid(True)
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
    fig.autofmt_xdate()
    _apply_styles(ax)

    fig.savefig(save_path, format='png', dpi=100)


In [25]:
def plot_line_defects(data, period, save_path='line_defects.png'):
    start_date = pd.to_datetime(period[0], dayfirst=True)
    end_date = pd.to_datetime(period[1], dayfirst=True)

    data = data.copy()
    data['Дата'] = pd.to_datetime(data['Дата'], errors='coerce')

    filtered = data[(data['Дата'] >= start_date) & (data['Дата'] <= end_date)]

    grouped = filtered.groupby('Дата')[['Количество', 'кол-во брака']].sum()
    grouped['% брака'] = grouped.apply(
        lambda row: (row['кол-во брака'] / row['Количество']) * 100 if row['Количество'] else 0,
        axis=1
    )

    total_qty = grouped['Количество'].sum()
    mean_value = (grouped['кол-во брака'].sum() / total_qty * 100) if total_qty else 0

    fig = Figure(figsize=(20, 7))
    ax = fig.subplots()
    ax.plot(grouped.index, grouped['% брака'], linestyle='-', color='orange', marker='o')
    ax.axhline(mean_value, color='red', linestyle='--', linewidth=2, label=f'Среднее: {mean_value:.2f}%')
    ax.set_title(f'% брака: {start_date.strftime("%d.%m.%y")} — {end_date.strftime("%d.%m.%y")}')
    ax.set_xlabel('Дата')
    ax.set_ylabel('% брака')
    ax.grid(True)
    ax.legend()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
    fig.autofmt_xdate()
    _apply_styles(ax)

    fig.savefig(save_path, format='png', dpi=100)


In [27]:
def bar_plot_defects(data, period, save_path='bar_defects.png'):
    defect_cols = [col for col in data.columns if
                   col.startswith("Виды брака") or col in ["Брак без видимых причин", "Излишки"]]

    start_date = pd.to_datetime(period[0], dayfirst=True)
    end_date = pd.to_datetime(period[1], dayfirst=True)

    data = data.copy()
    data['Дата'] = pd.to_datetime(data['Дата'], errors='coerce')

    filtered = data[(data['Дата'] >= start_date) & (data['Дата'] <= end_date)]

    grouped = filtered[defect_cols].sum()
    grouped = pd.to_numeric(grouped, errors="coerce").fillna(0).astype(int)
    grouped = grouped[grouped > 0].sort_values(ascending=False)

    labels = [col.replace('Виды брака_', '') for col in grouped.index]

    fig = Figure(figsize=(20, 8))
    ax = fig.subplots()
    ax.grid(True)
    ax.bar(labels, grouped.values, color='skyblue')
    ax.set_title(f'Типы дефектов: {start_date.strftime("%d.%m.%y")} — {end_date.strftime("%d.%m.%y")}')
    ax.set_xlabel('Тип дефекта')
    ax.set_ylabel('Количество')
    ax.tick_params(axis='x', rotation=60)
    fig.tight_layout()
    _apply_styles(ax)

    fig.savefig(save_path, format='png', dpi=100)


In [29]:
def pie_plot_defects(data, period, save_path='pie_defects.png'):
    defect_cols = [col for col in data.columns if
                   col.startswith("Виды брака") or col in ["Брак без видимых причин", "Излишки"]]

    start_date = pd.to_datetime(period[0], dayfirst=True)
    end_date = pd.to_datetime(period[1], dayfirst=True)

    data = data.copy()
    data['Дата'] = pd.to_datetime(data['Дата'], errors='coerce')

    filtered = data[(data['Дата'] >= start_date) & (data['Дата'] <= end_date)]

    grouped = filtered[defect_cols].sum()
    grouped = pd.to_numeric(grouped, errors="coerce").fillna(0).astype(int)
    grouped_nonzero = grouped[grouped > 0]

    fig = Figure(figsize=(12, 12))
    ax = fig.subplots()

    if grouped_nonzero.empty:
        ax.text(0.5, 0.5, 'Нет данных за указанный период', fontsize=16, ha='center')
        ax.axis('off')
    else:
        labels = [col.replace('Виды брака_', '') for col in grouped_nonzero.index]
        ax.pie(grouped_nonzero.values, labels=labels, autopct='%1.1f%%', startangle=140)
        ax.set_title(f'Структура дефектов: {start_date.strftime("%d.%m.%y")} — {end_date.strftime("%d.%m.%y")}', fontsize=16)

    fig.tight_layout()
    fig.savefig(save_path, format='png', dpi=100)


In [31]:
df = pd.read_csv('/Users/konstantin/Desktop/Рабочий стол — Konstantin’s MacBook Air/IT/PyCharm/dashboard_project/app/data/card_full_data.csv')

In [34]:
os.makedirs('output', exist_ok=True)
plot_line_total(df, ['01.09.2025', '15.09.2025'], save_path='output/line_total.png')
plot_line_defects(df, ['01.09.2025', '15.09.2025'], save_path='output/line_defects.png')
bar_plot_defects(df, ['01.09.2025', '15.09.2025'], save_path='output/bar_defects.png')
pie_plot_defects(df, ['01.09.2025', '15.09.2025'], save_path='output/pie_defects.png')

In [36]:
def get_kpis_table(data, period, save_path='output/kpis.xlsx'):
    start = pd.to_datetime(period[0], dayfirst=True)
    end = pd.to_datetime(period[1], dayfirst=True)

    # Копия данных и преобразование даты
    data = data.copy()
    data['Дата'] = pd.to_datetime(data['Дата'], errors='coerce')

    # Фильтрация по дате
    filtered = data[(data['Дата'] >= start) & (data['Дата'] <= end)].copy()

    # Приведение числовых колонок
    for col in ['Количество', 'кол-во брака', 'Стоимость брака']:
        filtered[col] = pd.to_numeric(filtered[col], errors='coerce').fillna(0)

    # Исключение строк без выпуска
    filtered = filtered[filtered['Количество'] > 0]

    # Расчёт KPI
    total = filtered['Количество'].sum()
    defects = filtered['кол-во брака'].sum()
    percent = (defects / total * 100) if total > 0 else 0
    total_money = filtered['Стоимость брака'].sum()

    # Формируем DataFrame с результатами
    kpi_df = pd.DataFrame([{
        'Период': f"{start.strftime('%d.%m.%Y')} — {end.strftime('%d.%m.%Y')}",
        'Всего вырублено': int(total),
        'Брак': int(defects),
        '% брака': round(percent, 2),
        'Сумма брака, ₽': round(total_money, 2)
    }])

    # Создание папки, если нужно
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Сохраняем в Excel (можно заменить на CSV при необходимости)
    kpi_df.to_excel(save_path, index=False)

    return kpi_df


In [38]:
kpi_table = get_kpis_table(df, ['01.09.2025', '15.09.2025'], save_path='output/kpis.xlsx')
print(kpi_table)

                    Период  Всего вырублено  Брак  % брака  Сумма брака, ₽
0  01.09.2025 — 15.09.2025           234505  5451     2.32       381070.86
